# Linear Regression: Training, Testing, and Predicting

# 1. General Setup

In [4]:
# Import analytics libraries
import pandas as pd
import numpy as np

# Import machine learning dependencies
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [5]:
# Raise errors, instead of give warnings
pd.set_option('mode.chained_Assignment', 'raise')

In [7]:
# Import data
import os
data_file_path = os.path.join('Data','stocks.csv')

# Preview data
stocks = pd.read_csv(data_file_path, index_col='Date')
display(stocks.head())
display(stocks.tail())

,Adj. Close,Open-Close Percent Change,High-Low Percent Change,Adj. Volume,Forecast
Date,,,,,
2004-08-19,50.322842,0.324968,8.441017,44659000.0,69.078238
2004-08-20,54.322689,7.227007,8.537313,22834300.0,67.839414
2004-08-23,54.869377,-1.227880,4.062357,18256100.0,68.912727
2004-08-24,52.597363,-5.726357,7.753210,15247300.0,70.668146
2004-08-25,53.164113,1.183658,3.966115,9188600.0,71.219849


,Adj. Close,Open-Close Percent Change,High-Low Percent Change,Adj. Volume,Forecast
Date,,,,,
2018-03-21,1094.00,0.130884,1.976619,1990515.0,NaN
2018-03-22,1053.15,-2.487014,3.265882,3418154.0,NaN
2018-03-23,1026.55,-2.360729,4.089299,2413517.0,NaN
2018-03-26,1054.09,0.332191,4.818025,3272409.0,NaN
2018-03-27,1006.94,-5.353887,6.707965,2940957.0,NaN
